<a href="https://colab.research.google.com/github/beachcities/TCE/blob/main/TCE_v0_1_Core_Prototype.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🏁 TCE v0.1 Final Report: 住所判定ロジックの検証結果

## 1. 検証の背景と課題
日本の住所データ（位置参照情報）と行政界ポリゴン（町丁目境界）を用いた住所判定エンジン「TCE」のプロトタイプ検証を行った。
当初の単純な「包含判定（Point in Polygon）」では精度 **99.31%** を記録したが、以下の課題が特定された。
* **道路・河川の空白問題:** e-Statポリゴンは街区のみを定義しており、道路や河川が含まれないため、道路上の座標が「壁の外（Out of Bounds）」と判定される。

## 2. アプローチの比較検証
課題解決のため、2つの幾何学的アプローチを比較検証した。

| アプローチ | 手法 | 結果 | 評価 |
| :--- | :--- | :--- | :--- |
| **A. バッファ法** | 壁を一律に太らせて隙間を埋める | **不採用** | 路地の隙間は埋まるが、大通り（幅員20m以上）の中心まで届かず。逆に広げすぎると隣接街区へ誤判定を起こすため、一律処理に限界あり。 |
| **B. ボロノイ法** | 最も近い壁を探す（最近傍判定） | **採用 (99.43%)** | 道路幅に関係なく、幾何学的に「中心線」で境界を引くのと同等の効果が得られた。 |

## 3. 最終結論 (Conclusion)
**「最近傍判定（Nearest Neighbor Logic）」** を採用することで、道路・河川上の点を含むすべての座標を救済することに成功した。

* **最終精度:** **99.4295%**
* **残存エラー:** 10件 (0.57%)
    * これらは全て「隣接する町丁目間での境界線上の微細なズレ（Border Jitter）」であり、建物の入り口位置と重心位置のズレや、測量時期の違いによるデータの誤差である。
* **判定:** TCE v0.1のロジックは実用上の理論値（Theoretical Limit）に到達したと判断し、本検証を完了とする。

In [ ]:
# === CELL 4: Buffer Optimizer (Metric Experiment) ===
import os
import glob
import pandas as pd
import geopandas as gpd
import zipfile
import warnings

warnings.filterwarnings('ignore')
BASE_PATH = "/content/drive/MyDrive/0_Professional/TCE_Project/data"

def run_optimizer_metric():
    print("📊 Starting Buffer Optimization (Metric System Check)...")
    if not os.path.exists(BASE_PATH): return

    # --- ファイル特定 ---
    files = os.listdir(BASE_PATH)
    shp_info = ref_info = None
    for f in files:
        if not f.lower().endswith(".zip"): continue
        full_path = os.path.join(BASE_PATH, f)
        try:
            with zipfile.ZipFile(full_path, 'r') as z:
                has_shp = any(n.lower().endswith('.shp') for n in z.namelist())
                csv_files = [n for n in z.namelist() if n.lower().endswith('.csv') and not n.startswith('__')]
                if has_shp and (shp_info is None) and ("13101" in f or "r2ka" in f or "A002" in f): shp_info = full_path
                if len(csv_files) > 0 and (ref_info is None): ref_info = (full_path, csv_files[0])
        except: continue

    if not shp_info or not ref_info: return

    # --- データロード ---
    gdf_bound = gpd.read_file(f"zip://{shp_info}", encoding='cp932', engine='pyogrio')
    gdf_bound = gdf_bound[['KEY_CODE', 'S_NAME', 'geometry']]
    with zipfile.ZipFile(ref_info[0], 'r') as z:
        with z.open(ref_info[1]) as f: df_ref = pd.read_csv(f, encoding='cp932')
    gdf_ref = gpd.GeoDataFrame(df_ref, geometry=gpd.points_from_xy(df_ref['経度'], df_ref['緯度']), crs="EPSG:6668")

    # --- メートル単位変換 (EPSG:6677) ---
    print("🌐 Converting to Meters (EPSG:6677)...")
    gdf_bound = gdf_bound.to_crs("EPSG:6677")
    gdf_ref = gdf_ref.to_crs("EPSG:6677")

    # --- 実験ループ ---
    buffer_sizes = [0.0, 5.0, 10.0]
    total_points = len(gdf_ref)

    print(f"   Target Points: {total_points}")
    print("   --------------------------------------")
    print("   | Buffer | Accuracy  | Errors |")
    print("   --------------------------------------")

    for buf in buffer_sizes:
        gdf_test = gdf_bound.copy()
        if buf > 0: gdf_test['geometry'] = gdf_test.geometry.buffer(buf)

        joined = gpd.sjoin(gdf_ref, gdf_test, how="left", predicate="within")
        town_col = '大字_丁目名' if '大字_丁目名' in joined.columns else '住居表示住所'
        joined['is_match'] = joined.apply(lambda r: r['S_NAME'] in str(r[town_col]) if pd.notnull(r['S_NAME']) else False, axis=1)

        # 重複排除後の正答率
        point_results = joined.groupby(joined.index)['is_match'].any()
        acc = point_results.mean() * 100
        error_count = total_points - point_results.sum()

        print(f"   | {buf:4.1f}m  | {acc:.4f}%  | {error_count:4d}   |")
    print("   --------------------------------------")

if __name__ == "__main__":
    run_optimizer_metric()

## 4. 解決策：ボロノイ分割 / 最近傍判定 (The Voronoi Logic)

バッファ法（壁の拡大）では、「広い道路（幅員20m超）」と「隣接する狭い路地」の両立が困難であることが判明しました。
そこで、幾何学的アプローチを**「包含判定（Point in Polygon）」から「最近傍判定（Nearest Neighbor）」へ転換**します。

### ロジックの概要
* **原理:** 全ての点について、物理的に**「最も距離が近い壁（ポリゴン）」**を探し出し、その住所を採用します。
* **効果:** これにより、道路の幅が何メートルあろうとも、数学的に**「道路の中心線」**で境界を引いたのと全く同じ結果が得られます。
* **実装:** 座標系をメートル単位（EPSG:6677）に変換した上で、`sjoin_nearest` アルゴリズムを適用します。

In [ ]:
# === CELL 5: Nearest Neighbor "Voronoi" Logic (Final Solution) ===
import os
import glob
import pandas as pd
import geopandas as gpd
import zipfile
import warnings

warnings.filterwarnings('ignore')
BASE_PATH = "/content/drive/MyDrive/0_Professional/TCE_Project/data"

def run_nearest_neighbor():
    print("💎 Starting Nearest Neighbor (Voronoi) Processing...")
    if not os.path.exists(BASE_PATH): return

    # --- ファイル特定 ---
    files = os.listdir(BASE_PATH)
    shp_info = ref_info = None
    for f in files:
        if not f.lower().endswith(".zip"): continue
        full_path = os.path.join(BASE_PATH, f)
        try:
            with zipfile.ZipFile(full_path, 'r') as z:
                has_shp = any(n.lower().endswith('.shp') for n in z.namelist())
                csv_files = [n for n in z.namelist() if n.lower().endswith('.csv') and not n.startswith('__')]
                if has_shp and (shp_info is None) and ("13101" in f or "r2ka" in f or "A002" in f): shp_info = full_path
                if len(csv_files) > 0 and (ref_info is None): ref_info = (full_path, csv_files[0])
        except: continue

    if not shp_info or not ref_info: return
    print(f"   SHP: {os.path.basename(shp_info)}")

    # --- データロード & 投影変換 ---
    gdf_bound = gpd.read_file(f"zip://{shp_info}", encoding='cp932', engine='pyogrio')
    gdf_bound = gdf_bound[['KEY_CODE', 'S_NAME', 'geometry']]
    with zipfile.ZipFile(ref_info[0], 'r') as z:
        with z.open(ref_info[1]) as f: df_ref = pd.read_csv(f, encoding='cp932')
    gdf_ref = gpd.GeoDataFrame(df_ref, geometry=gpd.points_from_xy(df_ref['経度'], df_ref['緯度']), crs="EPSG:6668")

    print("🌐 Converting to Metric System (EPSG:6677)...")
    gdf_bound = gdf_bound.to_crs("EPSG:6677")
    gdf_ref = gdf_ref.to_crs("EPSG:6677")

    # --- コア処理: 最近傍結合 ---
    print("⚡ Calculating Nearest Polygons...")
    joined = gpd.sjoin_nearest(gdf_ref, gdf_bound, how="left", distance_col="dist_m")

    # --- 評価 ---
    town_col = '大字_丁目名' if '大字_丁目名' in joined.columns else '住居表示住所'
    joined['is_match'] = joined.apply(lambda r: r['S_NAME'] in str(r[town_col]), axis=1)

    total = len(joined)
    correct = joined['is_match'].sum()
    error_count = total - correct
    accuracy = (correct / total) * 100

    print(f"\n=== 🏆 Final Result ===")
    print(f"   Accuracy:       {accuracy:.4f}%")
    print(f"   Errors:         {error_count}")

    # 救済された点（壁から離れていたが正解した点）
    rescued = joined[(joined['dist_m'] > 0) & (joined['is_match'])]
    if len(rescued) > 0:
        print(f"\n✨ Rescued Points (Road/Gap logic works): {len(rescued)}")
        print(rescued[[town_col, 'S_NAME', 'dist_m']].sort_values('dist_m', ascending=False).head(3))

    # 残ったエラー
    if error_count > 0:
        print(f"\n⚠️ Remaining Errors (Border Jitter): {error_count}")
        print(joined[~joined['is_match']][[town_col, 'S_NAME', 'dist_m']].head(3))

if __name__ == "__main__":
    run_nearest_neighbor()

## 5. 最終監査：残存エラーの可視化 (Final Audit)

Voronoiロジックにより、道路上の「難民」は救済され、精度は **99.43%** に達しました。
最後に、それでも残った **約0.6%（10件前後）のエラー** の正体を突き止めます。

### 監査の目的
これらのエラーが「ロジックの欠陥」なのか、それとも「データの限界（Border Jitter）」なのかを確定させます。
地図上で確認し、エラー等の点が**「境界線ギリギリ（数メートル以内）」**に存在していれば、これは建物の入り口位置と重心位置のズレや測量誤差に起因するものであり、本エンジンの**理論的限界値（Theoretical Limit）**に到達したと結論付けます。

In [ ]:
# === CELL 6: Visualizing Final Errors (Border Jitter) ===
import os
import pandas as pd
import geopandas as gpd
import folium
import zipfile
import warnings
from google.colab import drive

warnings.filterwarnings('ignore')
BASE_PATH = "/content/drive/MyDrive/0_Professional/TCE_Project/data"

def visualize_final_errors():
    if not os.path.exists(BASE_PATH): return

    # ファイル特定とロード (CELL 5と同様のロジックを簡略化)
    files = os.listdir(BASE_PATH)
    shp_info = ref_info = None
    for f in files:
        if not f.lower().endswith(".zip"): continue
        full_path = os.path.join(BASE_PATH, f)
        try:
            with zipfile.ZipFile(full_path, 'r') as z:
                has_shp = any(n.lower().endswith('.shp') for n in z.namelist())
                csv_files = [n for n in z.namelist() if n.lower().endswith('.csv') and not n.startswith('__')]
                if has_shp and (shp_info is None) and ("13101" in f or "r2ka" in f or "A002" in f): shp_info = full_path
                if len(csv_files) > 0 and (ref_info is None): ref_info = (full_path, csv_files[0])
        except: continue

    if not shp_info or not ref_info: return

    # Load & Project
    gdf_bound = gpd.read_file(f"zip://{shp_info}", encoding='cp932', engine='pyogrio')
    gdf_bound = gdf_bound[['KEY_CODE', 'S_NAME', 'geometry']]
    with zipfile.ZipFile(ref_info[0], 'r') as z:
        with z.open(ref_info[1]) as f: df_ref = pd.read_csv(f, encoding='cp932')
    gdf_ref = gpd.GeoDataFrame(df_ref, geometry=gpd.points_from_xy(df_ref['経度'], df_ref['緯度']), crs="EPSG:6668")

    gdf_bound_m = gdf_bound.to_crs("EPSG:6677")
    gdf_ref_m = gdf_ref.to_crs("EPSG:6677")

    # Nearest Join
    joined = gpd.sjoin_nearest(gdf_ref_m, gdf_bound_m, how="left", distance_col="dist_m")

    # Identify Errors
    town_col = '大字_丁目名' if '大字_丁目名' in joined.columns else '住居表示住所'
    joined['is_match'] = joined.apply(lambda r: r['S_NAME'] in str(r[town_col]), axis=1)
    errors = joined[~joined['is_match']].copy()

    print(f"🔍 Plotting {len(errors)} error points...")

    # Map (WGS84)
    gdf_bound_wgs = gdf_bound.to_crs(epsg=4326)
    errors_wgs = errors.to_crs(epsg=4326)

    if len(errors_wgs) > 0:
        center = errors_wgs.geometry.iloc[0]
        m = folium.Map(location=[center.y, center.x], zoom_start=16, tiles='OpenStreetMap')

        folium.GeoJson(
            gdf_bound_wgs,
            style_function=lambda x: {'color': 'blue', 'weight': 1, 'fillOpacity': 0.05},
            tooltip=folium.GeoJsonTooltip(fields=['S_NAME'])
        ).add_to(m)

        for idx, row in errors_wgs.iterrows():
            folium.Marker(
                location=[row.geometry.y, row.geometry.x],
                popup=f"<b>CONFLICT</b><br>True: {row[town_col]}<br>TCE: {row['S_NAME']}<br>Dist: {row['dist_m']:.2f}m",
                icon=folium.Icon(color='red', icon='exclamation', prefix='fa')
            ).add_to(m)

        display(m)

if __name__ == "__main__":
    visualize_final_errors()